### COMP4432 Machine Learning
### Assignment 5
### Megan Hoeksema

#### Packages

In [100]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.metrics import mean_squared_error, r2_score

#### Assignment 5, Part 1: Implement a Perceptron
Given the diabetes dataset you used during Assignment 2, implement an MLP Regressor.

In [76]:
# Load the dataset
diabetes = load_diabetes()
diabetesDF = pd.DataFrame(diabetes.data, columns = diabetes.feature_names)
diabetes['target'] = diabetes.target
diabetesDF['target'] = diabetes.target
diabetesDF.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [77]:
# Split the dataset
X = diabetesDF.drop(columns = ['target'])
y = diabetesDF[['target']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
print("Sizes of training and test sets:", [len(elem) for elem in [X_train, X_test, y_train, y_test]])

Sizes of training and test sets: [353, 89, 353, 89]


In [78]:
# MLP Regressor
mlp = MLPRegressor(random_state=1, max_iter=10000).fit(X_train, y_train)
mlpPreds = mlp.predict(X_train)
mlp.score(X_test, y_test)

/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.4286231109167039

#### Assignment 5, Part 2: Implement a Keras Classifier
Given the prepared Titanic dataset from Assignment 3, implement a Keras sequential classifier with relu activation functions.

In [79]:
# Load the dataset
titanic_DF = pd.read_csv('titanic_train.csv')
titanic_DF.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [80]:
# Data Preprocessing
def impute_age(columns):
    Age = columns[0]
    Pclass = columns[1]
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        elif Pclass == 3:
            return 24
    else:
        return Age

titanic_DF['Age'] = titanic_DF[['Age','Pclass']].apply(impute_age,axis=1)

titanic_DF = pd.get_dummies(titanic_DF, columns=['Sex', 'Embarked'])
titanic_DF = titanic_DF.drop(columns = ['PassengerId',  'Ticket', 'Name', 'Cabin'])

titanic_DF.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [81]:
# Splitting the data
X = titanic_DF.drop(columns = ['Survived'])
y = titanic_DF['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
print("Sizes of training and test sets:", [len(elem) for elem in [X_train, X_test, y_train, y_test]])

Sizes of training and test sets: [712, 179, 712, 179]


In [82]:
x = np.asarray(X_train).astype('float32')

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

model.fit(X_train, y_train, epochs=10)

Epoch 1/10
23/23 [==============================] - 0s 2ms/step - loss: 18.1456 - mae: 2.1518
Epoch 2/10
23/23 [==============================] - 0s 2ms/step - loss: 1.6071 - mae: 0.9233
Epoch 3/10
23/23 [==============================] - 0s 2ms/step - loss: 0.5216 - mae: 0.5534
Epoch 4/10
23/23 [==============================] - 0s 2ms/step - loss: 0.3609 - mae: 0.4690
Epoch 5/10
23/23 [==============================] - 0s 2ms/step - loss: 0.2947 - mae: 0.4293
Epoch 6/10
23/23 [==============================] - 0s 2ms/step - loss: 0.2555 - mae: 0.4084
Epoch 7/10
23/23 [==============================] - 0s 2ms/step - loss: 0.2348 - mae: 0.3917
Epoch 8/10
23/23 [==============================] - 0s 3ms/step - loss: 0.2215 - mae: 0.3885
Epoch 9/10
23/23 [==============================] - 0s 2ms/step - loss: 0.2104 - mae: 0.3658
Epoch 10/10
23/23 [==============================] - 0s 2ms/step - loss: 0.2022 - mae: 0.3685


In [83]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

6/6 - 0s - loss: 0.2189 - mae: 0.3866 - 77ms/epoch - 13ms/step

Test accuracy: 0.3865755498409271


#### Assignment 5, Part 3: Implement a Keras Regressor
Given the prepared bike-share dataset from Assignment 4, implement a Keras sequential regressor with relu activation functions.

In [84]:
# Load the dataset
bikeDF = pd.read_csv('bike_share_hour.csv')
bikeDF = bikeDF.astype({"season":'category', "workingday":'category', "weathersit":'category', 'dteday':'category',
                        'yr':'category', 'mnth':'category','hr':'category','holiday':'category','weekday':'category' })
scaler = StandardScaler()
numeric_cols = list(bikeDF.dtypes[bikeDF.dtypes != 'category'].index)
bikeDF.loc[:,numeric_cols] = scaler.fit_transform(bikeDF.loc[:,numeric_cols])
bikeDF = bikeDF.drop(columns = ['casual', 'registered', 'dteday', 'instant'])
bikeDF.head()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,0,1,0,0,6,0,1,-1.334648,-1.093281,0.947372,-1.553889,-0.956339
1,1,0,1,1,0,6,0,1,-1.438516,-1.181732,0.895539,-1.553889,-0.824022
2,1,0,1,2,0,6,0,1,-1.438516,-1.181732,0.895539,-1.553889,-0.868128
3,1,0,1,3,0,6,0,1,-1.334648,-1.093281,0.636370,-1.553889,-0.972879
4,1,0,1,4,0,6,0,1,-1.334648,-1.093281,0.636370,-1.553889,-1.039037


In [85]:
# Splitting the data
X = bikeDF.drop(columns = ['cnt'])
y = bikeDF['cnt']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
print("Sizes of training and test sets:", [len(elem) for elem in [X_train, X_test, y_train, y_test]])

Sizes of training and test sets: [13903, 3476, 13903, 3476]


In [86]:
x = np.asarray(X_train).astype('float32')

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

model.fit(X_train, y_train, epochs=10)

Epoch 1/10
435/435 [==============================] - 1s 1ms/step - loss: 0.6893 - mae: 0.6013
Epoch 2/10
435/435 [==============================] - 1s 1ms/step - loss: 0.4650 - mae: 0.4957
Epoch 3/10
435/435 [==============================] - 1s 1ms/step - loss: 0.4241 - mae: 0.4671
Epoch 4/10
435/435 [==============================] - 1s 1ms/step - loss: 0.4070 - mae: 0.4539
Epoch 5/10
435/435 [==============================] - 1s 1ms/step - loss: 0.3951 - mae: 0.4452
Epoch 6/10
435/435 [==============================] - 1s 1ms/step - loss: 0.3790 - mae: 0.4336
Epoch 7/10
435/435 [==============================] - 1s 1ms/step - loss: 0.3729 - mae: 0.4335
Epoch 8/10
435/435 [==============================] - 1s 1ms/step - loss: 0.3634 - mae: 0.4258
Epoch 9/10
435/435 [==============================] - 1s 1ms/step - loss: 0.3489 - mae: 0.4131
Epoch 10/10
435/435 [==============================] - 1s 1ms/step - loss: 0.3405 - mae: 0.4078


In [87]:
test_loss, test_mae = model.evaluate(X_test,  y_test, verbose=2)
print('\nTest accuracy:', test_mae)

109/109 - 0s - loss: 0.3211 - mae: 0.4043 - 155ms/epoch - 1ms/step

Test accuracy: 0.40427565574645996


#### Assignment 5, Part 4: Tune Your Keras Regressor
Tune your Keras regressor from Part 3 by implementing a grid search with different optimizers.

In [104]:
reg = KerasRegressor(model = model, optimizer = 'adam', loss = 'mean_squared_error', batch_size = 10,
                    epochs = 100, verbose = 0)

batch_size = [25]
epochs = [10]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(batch_size = batch_size, epochs = epochs, optimizer = optimizer)

grid_search = GridSearchCV(estimator = reg, param_grid = param_grid)

grid_model = grid_search.fit(X_train, y_train)

INFO:tensorflow:Assets written to: ram:///tmp/tmp6qr1l6k1/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpz3r15vhs/assets


/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/scikeras/wrappers.py", line 171

INFO:tensorflow:Assets written to: ram:///tmp/tmplo1pjjqd/assets


/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/scikeras/wrappers.py", line 171

INFO:tensorflow:Assets written to: ram:///tmp/tmpqt0h1es8/assets


...

INFO:tensorflow:Assets written to: ram:///tmp/tmpnio752x9/assets


/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/scikeras/wrappers.py", line 171

INFO:tensorflow:Assets written to: ram:///tmp/tmpqauy68m4/assets


/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/scikeras/wrappers.py", line 171

INFO:tensorflow:Assets written to: ram:///tmp/tmpj3pgvnyx/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpakg1rz_e/assets
